In [10]:
import z_analysis_conf, json, time, math, statistics
from boltiot import Sms, Bolt

In [11]:
def compute_bounds(history_data,frame_size,factor):
    if len(history_data)<frame_size :
        return None
    if len(history_data)>frame_size :
        del history_data[0:len(history_data)-frame_size]
    Mn=statistics.mean(history_data)
    Variance=0
    for data in history_data :
        Variance += math.pow((data-Mn),2)
    Zn = factor * math.sqrt(Variance / frame_size)
    High_bound = history_data[frame_size-1]+Zn
    Low_bound = history_data[frame_size-1]-Zn
    return [High_bound,Low_bound]

In [12]:
mybolt = Bolt(z_analysis_conf.API_KEY, z_analysis_conf.DEVICE_ID)
sms = Sms(z_analysis_conf.SSID, z_analysis_conf.AUTH_TOKEN, z_analysis_conf.TO_NUMBER, z_analysis_conf.FROM_NUMBER)
history_data=[]

In [ ]:
while True:
    response = mybolt.analogRead('A0')

    data = json.loads(response)
    if data['success'] != 1:
        print("There was an error while retriving the data.")
        print("This is the error:"+data['value'])
        time.sleep(5)
        continue
    print ("This is the value: "+data['value'])
    
    sensor_value=0
    try:
        sensor_value = int(data['value'])
    except e:
        print("There was an error while parsing the response: ",e)
        continue

    bound = compute_bounds(history_data,z_analysis_conf.FRAME_SIZE,z_analysis_conf.MUL_FACTOR)
    if not bound:
        required_data_count=z_analysis_conf.FRAME_SIZE-len(history_data)
        print("Not enough data to compute Z-score. Need ",required_data_count," more data points")
        history_data.append(int(data['value']))
        time.sleep(5)
        continue

    try:
        if sensor_value > bound[0] :
            print ("The light level increased suddenly. Sending an SMS")
            response = sms.send_sms("Someone turned on the lights")
            print("This is the response ",response)
        elif sensor_value < bound[1]:
            print ("The light level decreased suddenly. Sending an SMS")
            response = sms.send_sms("Someone turned off the lights")
            print("This is the response ",response)
        history_data.append(sensor_value);
    except Exception as e:
        print ("Error",e)
    time.sleep(5)

This is the value: 512
Not enough data to compute Z-score. Need  10  more data points
This is the value: 550
Not enough data to compute Z-score. Need  9  more data points
This is the value: 509
Not enough data to compute Z-score. Need  8  more data points
This is the value: 525
Not enough data to compute Z-score. Need  7  more data points
This is the value: 550
Not enough data to compute Z-score. Need  6  more data points
This is the value: 541
Not enough data to compute Z-score. Need  5  more data points
This is the value: 526
Not enough data to compute Z-score. Need  4  more data points
This is the value: 502
Not enough data to compute Z-score. Need  3  more data points
This is the value: 550
Not enough data to compute Z-score. Need  2  more data points
This is the value: 502
Not enough data to compute Z-score. Need  1  more data points
This is the value: 550
This is the value: 553
This is the value: 547
This is the value: 550
This is the value: 516
This is the value: 534
This is the